In [167]:
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import RMSprop
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [168]:
training_df = pd.read_csv('nn.csv', index_col=0)
training_df.drop(columns=[training_df.columns[0], training_df.columns[1], training_df.columns[2], training_df.columns[3], training_df.columns[len(training_df.columns)-1]], axis=1,  inplace=True)

# Fill in missing
def fillMissing(df, feature, method):
  if method == "mode":
    df[feature] = df[feature].fillna(df[feature].mode()[0])
  elif method == "median":
    df[feature] = df[feature].fillna(df[feature].median())
  else:
    df[feature] = df[feature].fillna(df[feature].mean())

features_missing = training_df.columns[training_df.isna().any()]
for feature in features_missing:
  fillMissing(training_df, feature = feature, method = "mean")


cols_at_end = ['Analyst']
training_df = training_df[[c for c in training_df if c not in cols_at_end] 
        + [c for c in cols_at_end if c in training_df]]
encoder = LabelEncoder()
encoder.fit(training_df['Analyst'])
encoder_values = encoder.transform(training_df['Analyst'])
training_df['analyst_predictions'] = encoder_values
training_df.drop(['Analyst'], axis=1, inplace=True)
training_df.dropna(inplace=True)

In [169]:
training_df

,headline_polarity,convo_polarity,beta,profitMargins,forwardEps,bookValue,heldPercentInstitutions,shortRatio,shortPercentOfFloat,analyst_predictions
0,0.094626,0.191632,1.734251,0.050000,2.790000,22.710000,1.000000,3.080000,0.030000,0
1,0.250000,0.210000,1.300000,0.060000,12.760000,52.440000,1.030000,6.920000,0.070000,0
2,0.094626,0.191632,1.734251,0.009524,2.948967,129.998838,0.648889,4.685499,0.063772,0
3,0.094626,0.191632,1.410000,0.920000,3.830000,16.070000,0.140000,4.980000,0.010000,0
4,0.094626,0.191632,1.080000,0.060000,1.560000,7.000000,0.060000,1.390000,0.063772,0
...,...,...,...,...,...,...,...,...,...,...
4320,0.000000,0.030000,1.734251,0.009524,2.948967,129.998838,0.648889,4.685499,0.063772,4
4321,0.180000,0.130000,0.290000,-0.570000,0.450000,16.680000,0.010000,0.190000,0.063772,4
4322,0.200000,0.040000,1.734251,0.009524,2.948967,129.998838,0.648889,4.685499,0.063772,4
4323,0.350000,0.210000,1.734251,0.070000,1.000000,1.970000,0.140000,3.930000,0.063772,4


In [170]:
X_total = training_df[[x for x in training_df.columns if x not in ['analyst_predictions']]]
y_total = training_df['analyst_predictions']
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.33, random_state=42)

In [174]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(9,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [175]:
history = model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=100,
    verbose=0,
    validation_data=(X_test, y_test))

In [176]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.0480068922042847
Test accuracy: 0.6022409200668335
